In [ ]:
import gym
import math
import random
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

 Description:
        The agent (a car) is started at the bottom of a valley. For any given
        state the agent may choose to accelerate to the left, right or cease
        any acceleration.
    Source:
        The environment appeared first in Andrew Moore's PhD Thesis (1990).
    Observation:
        Type: Box(2)
        Num    Observation               Min            Max
        0      Car Position              -1.2           0.6
        1      Car Velocity              -0.07          0.07
    Actions:
        Type: Discrete(3)
        Num    Action
        0      Accelerate to the Left
        1      Don't accelerate
        2      Accelerate to the Right
        Note: This does not affect the amount of velocity affected by the
        gravitational pull acting on the car.
    Reward:
         Reward of 0 is awarded if the agent reached the flag (position = 0.5)
         on top of the mountain.
         Reward of -1 is awarded if the position of the agent is less than 0.5.
    Starting State:
         The position of the car is assigned a uniform random value in
         \[-0.6 , -0.4].
         The starting velocity of the car is always assigned to 0.
    Episode Termination:
         The car position is more than 0.5
         Episode length is greater than 200
         
Note : This implementation also has a custom reward function which returns the reward according to position of the car.
        and two different ways to for epsilon decay.

In [ ]:
env = gym.make('MountainCar-v0')
env.reset()

def newreward(pos):
    if(pos >= 0.5):
        #return 2
        return 0
    else:
        #return (pos-(-1.2)/1.8- 1
        return -1

def QLearning(env, learning, epsilon, min_eps, episodes):
    num_states = (env.observation_space.high - env.observation_space.low)*np.array([10, 50])
    num_states = np.round(num_states, 0).astype(int) + 1
    
    Q = np.random.uniform(low = -1, high = 0, size = (num_states[0], num_states[1], env.action_space.n))
    
    reward_list = []
    average_reward_list= []
    
    reduction =  (epsilon - min_eps)/episodes 
    
    eps1 = epsilon
    
    first = episodes + 1
    
    for i in range (episodes):
        done = False
        tot_reward = 0
        reward = 0
        state = env.reset()
        
        state_adj = (state - env.observation_space.low)*np.array([10, 50])
        state_adj= np.round(state_adj, 0).astype(int)
        
        while done != True : 
            if i >= (episodes - 5) or i<5:
                env.render()
                
            if np.random.random()< 1 - epsilon:
                action = np.argmax(Q[state_adj[0], state_adj[1]])
            else:
                action = np.random.randint(0, env.action_space.n)
                
            state2, reward, done, info = env.step(action)
            
            state2_adj = (state2 - env.observation_space.low)*np.array([10,50])
            state2_adj = np.round(state2_adj, 0).astype(int)
                            
            if done and state2[0] >= 0.5:
                    Q[state_adj[0], state_adj[1], action] = reward
            else:
                    delta = learning*(newreward(state[0]) + np.max(Q[state2_adj[0], state2_adj[1]] - Q[state_adj[0], state_adj[1], action]))
                    Q[state_adj[0], state_adj[1], action] += delta
            
            if state2[0] >= 0.5 and i<first:
                    first = i
                    print('First Clear on episode {}'.format(i+1))
                                                  
            tot_reward += newreward(state2[0])
            state_adj = state2_adj
                                                  
        if epsilon > min_eps:
                #epsilon -= reduction
                epsilon *= eps1
        
                            
        reward_list.append(tot_reward)
                                    
        if (i+1) % 100 == 0:
            average_reward = np.mean(reward_list)
            average_reward_list.append(average_reward)
            reward_list = []
                                                
        if (i+1) % 1000 == 0:
            print('Episode {} average reward: {}'.format(i+1, average_reward))
        
    env.close()
                
    return average_reward_list, Q
                                                  

In [ ]:
env.reset()
rewards, Qpts = QLearning(env, 0.2, 0.9, 0, 1000)

In [ ]:
plt.plot(100*(np.arange(len(rewards)) +1), rewards)
plt.xlabel('Episodes')
plt.ylabel('Average Reward')
plt.title('Average Reward vs Episodes')

In [ ]:
for i in range (20):
    done1 = False
    state1=env.reset()
    state_adj1 = (state1 - env.observation_space.low)*np.array([10, 50])
    state_adj1= np.round(state_adj1, 0).astype(int)
    while done1 != True:
        env.render()
        action = np.argmax(Qpts[state_adj1[0], state_adj1[1]])
        state3, reward1, done1, info1 = env.step(action)
        state3_adj = (state3 - env.observation_space.low)*np.array([10,50])
        state3_adj = np.round(state3_adj, 0).astype(int)
        state_adj1 = state3_adj

In [ ]:
env.close()